## 의약품 데이터 수집

의약품 데이터를 식약처 오픈 API(공공데이터포털)로 부터 수집한 뒤 csv로 저장한다.

### 라이브러리 import 및 환경설정

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import display
from tqdm.notebook import tqdm, trange
from pathlib import Path
import pandas as pd
import numpy as np
import urllib.request
import json

#### 1. 낱알식별정보 csv 파일 불러오기

In [ ]:
# /dat 디렉토리 내, 이름 형식이 yyyy-mm-dd_xxhxxmxxs 인 디렉토리 하나를 선택해야 함
import tkinter as tk
from tkinter import filedialog
window = tk.Tk()
window.file = filedialog.askdirectory()

filepath = f"{window.file}\\csv\\picked-out\\알약외형정보.csv"
phill_df = pd.read_csv(filepath, index_col=0)

phill_df.info()
display(phill_df.head())

window.destroy()

### 2. 알약 이미지 다운로드

#### 2.1. 디렉토리 생성

In [ ]:
import os
exists = os.path.exists

dir = f"{window.file}\\phill_img\\"
if not exists(dir):
    os.mkdir(dir)

#### 2.2. 이미지 다운로드 실행

In [ ]:
# 주피터 노트북 에서 이벤트 루프를 사용할 때 발생하는 에러 방지
import nest_asyncio
nest_asyncio.apply()

import asyncio
import urllib.request as urlreq

async def download_a_img(url, filepath):
    retry_cnt = 0
    retry_cap = 5
    while True:
        try:
            return await loop.run_in_executor(None, urlreq.urlretrieve, url, filepath)
        except Exception as e:
            print(f"이미지파일({filepath}) 다운로드 중 예외발생")
            print(f"=> {e}")
            if retry_cnt >= retry_cap:
                print(f"재시도 횟수 초과! 이벤트 루프를 중지합니다.")
                loop.stop()

async def download_imgs():
    tasks = (asyncio.create_task(download_a_img(row["ITEM_IMAGE"], dir + str(row["ITEM_SEQ"]) + "-" + str(idx) + ".jpg")) for idx, row in phill_df[["ITEM_SEQ", "ITEM_IMAGE"]].iterrows())
    res = await asyncio.gather(*tasks)
    return res

loop = asyncio.get_event_loop()
res = loop.run_until_complete(download_imgs())